# Traffic data aggregation using network distance metrics and cokriging
Final project for GEOG 8102  
Yue Lin (lin.3326 at osu.edu)  
Created: 4/14/2020

## Mapping points to non-Euclidean space

Read graph data

In [22]:
import csv
with open('d04_tomtom_od.csv', newline='') as csvfile:
    alllines = csv.reader(csvfile)
    graph = []
    for line in alllines:
        item = []
        item.append(int(line[2]))
        item.append(int(line[3]))
        item.append(float(line[5]))
        graph.append(item)
    print(graph[:5])

[[1, 1, 0.0], [1, 220, 3145.43243], [1, 290, 3189.500113], [1, 221, 6100.009007], [1, 328, 10813.73763]]


Costruct distance matrix

In [ ]:
b_matrix = [ [INF] * n for i in range(n)]
for i in n:
    for j in n:
        d = distance_matrix[i][j]
        b[i][j] = 1/2 * (-d*d + 1/n)